In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1713750/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_44_3,0.998373,0.642148,0.687141,0.996825,0.000624,0.212436,0.219164,0.001078,0.012105,0.024977,0.996721,0.025171,144.759222,331.148961,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_44_2,0.998372,0.644653,0.687138,0.996984,0.000624,0.210949,0.219166,0.001024,0.012107,0.024987,0.996719,0.025182,144.757570,331.147309,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_44_4,0.998372,0.642004,0.687134,0.996791,0.000624,0.212522,0.219169,0.001089,0.012100,0.024987,0.996719,0.025182,144.757561,331.147301,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_44_5,0.998370,0.641700,0.687574,0.996765,0.000625,0.212703,0.218861,0.001098,0.012096,0.025004,0.996714,0.025198,144.754944,331.144683,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_44_6,0.998349,0.641254,0.686976,0.996418,0.000633,0.212967,0.219280,0.001216,0.012091,0.025164,0.996672,0.025360,144.729389,331.119128,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,model_1_0_4,0.375264,0.256312,0.298616,0.193060,0.239592,0.441485,0.491334,0.260511,0.273389,0.489481,-0.259233,0.493290,132.857638,319.247378,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
496,model_1_0_3,0.236830,0.153386,0.198096,-0.000101,0.292683,0.502587,0.561751,0.322871,0.296412,0.541001,-0.538265,0.545212,132.457333,318.847073,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
497,model_1_0_2,0.140835,0.081761,0.186496,-0.027435,0.329497,0.545106,0.569876,0.331695,0.318471,0.574019,-0.731754,0.578486,132.220374,318.610113,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
498,model_1_0_1,0.013290,-0.006819,0.163660,-0.302298,0.378412,0.597691,0.585874,0.420431,0.334746,0.615152,-0.988836,0.619939,131.943544,318.333283,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
